In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
cust_features = pd.read_csv("features_data.csv")
cust_balances = pd.read_csv("equity_value_data.csv")
cust_features

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,user_id
0,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,33.129417,stock,40.0,med_time_horizon,895044c23edc821881e87da749c01034
1,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,16.573517,stock,200.0,short_time_horizon,458b1d95441ced242949deefe8e4b638
2,med_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,10.008367,stock,25.0,long_time_horizon,c7936f653d293479e034865db9bb932f
3,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,1.031633,stock,100.0,short_time_horizon,b255d4bd6c9ba194d3a350b3e76c6393
4,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,8.187250,stock,20.0,long_time_horizon,4a168225e89375b8de605cbc0977ae91
...,...,...,...,...,...,...,...,...,...
5579,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,8.339283,stock,300.0,long_time_horizon,03880c726d8a4e5db006afe4119ad974
5580,med_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,7.241383,stock,100.0,short_time_horizon,ae8315109657f44852b24c6bca4decd6
5581,med_risk_tolerance,no_investment_exp,very_important_liq_need,both,22.967167,stock,50.0,short_time_horizon,f29c174989f9737058fe808fcf264135
5582,med_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,10.338417,stock,100.0,long_time_horizon,24843497d1de88b2e7233f694436cb3a


In [3]:
cust_balances

,timestamp,close_equity,user_id
0,2016-11-16T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03
1,2016-11-17T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03
2,2016-11-18T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03
3,2016-11-21T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03
4,2016-11-22T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03
...,...,...,...
1119153,2017-08-14T00:00:00Z,2270.71,98580360c4fb5b0ec511cd87f0d429ed
1119154,2017-08-15T00:00:00Z,2275.02,98580360c4fb5b0ec511cd87f0d429ed
1119155,2017-08-16T00:00:00Z,2282.03,98580360c4fb5b0ec511cd87f0d429ed
1119156,2017-08-17T00:00:00Z,2237.45,98580360c4fb5b0ec511cd87f0d429ed


In [4]:
cust_balances.timestamp = cust_balances.timestamp.astype("datetime64[ns]")
end_date = cust_balances.timestamp.iloc[-1]
cust_balances = cust_balances.sort_values(by=["user_id", "timestamp"])
cust_balances.head(11)

,timestamp,close_equity,user_id
505994,2016-08-18,1211.6055,0012db34aa7b083f5714e7831195e54d
505995,2016-08-19,1173.5640,0012db34aa7b083f5714e7831195e54d
505996,2016-08-22,1253.0597,0012db34aa7b083f5714e7831195e54d
505997,2016-08-23,1252.9050,0012db34aa7b083f5714e7831195e54d
505998,2016-08-24,1262.1360,0012db34aa7b083f5714e7831195e54d
505999,2016-08-25,1226.4250,0012db34aa7b083f5714e7831195e54d
506000,2016-08-26,1227.9610,0012db34aa7b083f5714e7831195e54d
506001,2016-08-29,1250.9700,0012db34aa7b083f5714e7831195e54d
506002,2016-08-30,1248.1000,0012db34aa7b083f5714e7831195e54d
506003,2016-08-31,1241.5560,0012db34aa7b083f5714e7831195e54d


In [5]:
balance_stats = cust_balances[["user_id", "close_equity"]].groupby("user_id").agg(["count", "mean", "std", "min", "max"])
balance_stats.columns = balance_stats.columns.to_flat_index()
balance_stats

,"(close_equity, count)","(close_equity, mean)","(close_equity, std)","(close_equity, min)","(close_equity, max)"
user_id,,,,,
0012db34aa7b083f5714e7831195e54d,252,609.100576,191.975127,355.3723,1262.1360
001d6c77dbdb3213cead7673f250bfdc,252,3307.140361,449.330147,2693.0400,4030.4577
002e4653171ddc61c3cd30603cd7bd3e,128,117.203750,6.480472,100.0000,134.3800
00384fa9be6fdca1b786bae70d78f88f,124,462.786815,24.522490,414.4200,518.0000
0042aac295a0d4df88f4b83012778bd4,252,1561.098745,933.690027,123.5265,2559.1550
...,...,...,...,...,...
ff9ee08791e20724a86995ab2bc72578,252,92.444723,47.137114,57.7617,259.4046
ffa12d2f97e310910291f9b26fb2318d,252,6064.207556,1577.089485,2661.6500,8486.5050
ffae713096867a32e74f633060667153,224,20.166519,7.194193,16.5600,37.5200


In [6]:
set(cust_features.user_id) - set(balance_stats.index)

set()

In [7]:
cust_balances["time_gap"] = cust_balances.timestamp.diff(-1).dt.days.fillna(0).astype("int64") * -1
cust_balances

,timestamp,close_equity,user_id,time_gap
505994,2016-08-18,1211.6055,0012db34aa7b083f5714e7831195e54d,1
505995,2016-08-19,1173.5640,0012db34aa7b083f5714e7831195e54d,3
505996,2016-08-22,1253.0597,0012db34aa7b083f5714e7831195e54d,1
505997,2016-08-23,1252.9050,0012db34aa7b083f5714e7831195e54d,1
505998,2016-08-24,1262.1360,0012db34aa7b083f5714e7831195e54d,1
...,...,...,...,...
675754,2017-08-14,2156.2400,ffc1e622f3a0b2666f09a6dcb7f27918,1
675755,2017-08-15,2134.7100,ffc1e622f3a0b2666f09a6dcb7f27918,1
675756,2017-08-16,2152.1200,ffc1e622f3a0b2666f09a6dcb7f27918,1
675757,2017-08-17,2042.2800,ffc1e622f3a0b2666f09a6dcb7f27918,1


In [8]:
cust_balances[cust_balances.time_gap < 0]

,timestamp,close_equity,user_id,time_gap
506245,2017-08-17,487.7885,0012db34aa7b083f5714e7831195e54d,-364
417392,2017-08-17,3981.3350,001d6c77dbdb3213cead7673f250bfdc,-182
1010028,2017-08-18,134.3800,002e4653171ddc61c3cd30603cd7bd3e,-176
629750,2017-08-18,426.2400,00384fa9be6fdca1b786bae70d78f88f,-365
417644,2017-08-17,1769.2150,0042aac295a0d4df88f4b83012778bd4,-230
...,...,...,...,...
750576,2017-08-18,16790.4700,ff98cbc9633a9255a711d02ebc1e8294,-365
455492,2017-08-17,62.1951,ff9ee08791e20724a86995ab2bc72578,-365
226399,2017-08-16,7448.1895,ffa12d2f97e310910291f9b26fb2318d,-322
341092,2017-08-17,32.9300,ffae713096867a32e74f633060667153,-183


In [9]:
cust_balances.time_gap = cust_balances.apply(lambda x: (end_date - x.timestamp).days if x.time_gap < 0 else x.time_gap, axis=1)

In [10]:
cust_churns = cust_balances[cust_balances.time_gap >= 28]
cust_churns

,timestamp,close_equity,user_id,time_gap
152685,2017-02-27,59.4500,00440034cc4152bfb01b30f5c381c4e3,80
959321,2017-04-28,25.1400,004aab1640f3a04b87b1f404fb4c018d,112
1042837,2017-07-10,100.9800,004ea9d7662aa8dc840bbff212cfa4b8,39
293708,2016-11-04,178.8797,005d630a68b4ab3a2f4cd49d9a87c50d,123
161451,2017-05-19,1143.6600,00f89f56f25989b0bb7ea05bac2dccc4,91
...,...,...,...,...
955338,2017-04-03,575.3300,ff0ae95285c43e3a5af84860bffaa544,81
857073,2016-09-29,1001.2500,ff377467d4e28b425266a8b2c8b2f5c7,60
447016,2017-03-06,16.8000,ff6d64d75fa2ffd703dabf66b7b86b99,165
29165,2017-01-12,17.1700,ff7610fdd7ac5cbfa0b17aca53af5db4,172


In [11]:
cust_churn_count = cust_churns.user_id.value_counts()
cust_churn_count

50b846ab40078ae27b0bf7d6004c9a28    3
9121bbdb811bcb10de9b6f85b2b5e458    3
14b44db6ff01a66a284d6d4a62975706    3
4db4db8398496203a7def94692739f86    3
ec781fbc977db5d88af48e5304d5da41    3
                                   ..
57d9b22067d7b0312f0cd6ba73d44743    1
5813f69ba4430325260e394908a937ab    1
582e57386c12abb201c6d620786800d3    1
58756af245e132e09b91e4acfbcb8dd1    1
ff98cbc9633a9255a711d02ebc1e8294    1
Name: user_id, Length: 1003, dtype: int64

In [12]:
cust_features = cust_features.merge(balance_stats, left_on="user_id", right_index=True)
cust_features

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,user_id,"(close_equity, count)","(close_equity, mean)","(close_equity, std)","(close_equity, min)","(close_equity, max)"
0,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,33.129417,stock,40.0,med_time_horizon,895044c23edc821881e87da749c01034,190,77.118526,10.739414,38.6200,88.9000
1,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,16.573517,stock,200.0,short_time_horizon,458b1d95441ced242949deefe8e4b638,252,351.411468,26.057862,306.5100,394.9200
2,med_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,10.008367,stock,25.0,long_time_horizon,c7936f653d293479e034865db9bb932f,252,49.240000,0.000000,49.2400,49.2400
3,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,1.031633,stock,100.0,short_time_horizon,b255d4bd6c9ba194d3a350b3e76c6393,143,186.040280,11.967710,102.6200,212.4600
4,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,8.187250,stock,20.0,long_time_horizon,4a168225e89375b8de605cbc0977ae91,252,208.753365,146.789955,33.0300,483.7306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5579,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,8.339283,stock,300.0,long_time_horizon,03880c726d8a4e5db006afe4119ad974,252,2742.329510,205.873860,2307.0000,3096.7200
5580,med_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,7.241383,stock,100.0,short_time_horizon,ae8315109657f44852b24c6bca4decd6,99,310.029010,226.080679,46.9500,638.0800
5581,med_risk_tolerance,no_investment_exp,very_important_liq_need,both,22.967167,stock,50.0,short_time_horizon,f29c174989f9737058fe808fcf264135,197,116.134472,134.437240,37.4464,460.6100
5582,med_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,10.338417,stock,100.0,long_time_horizon,24843497d1de88b2e7233f694436cb3a,141,330.172163,48.174276,53.1900,374.2500


In [13]:
cust_features["churned"] = cust_features.apply(lambda x: 1 if x.user_id in cust_churn_count.index else 0, axis=1)
cust_features[cust_features.churned == 1]

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,user_id,"(close_equity, count)","(close_equity, mean)","(close_equity, std)","(close_equity, min)","(close_equity, max)",churned
5,med_risk_tolerance,no_investment_exp,not_important_liq_need,iOS,12.968117,stock,100.0,med_time_horizon,0322b1d744cc89ebbe6ebe70d824d922,53,413.583387,262.036251,95.7800,841.41,1
6,low_risk_tolerance,good_investment_exp,very_important_liq_need,Android,30.710083,stock,5.0,long_time_horizon,f4cac8c1544b723ddedc9bdba81c64c0,225,364.784314,358.654484,13.7400,1347.68,1
12,high_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,74.346767,stock,200.0,short_time_horizon,b63f1ab8005b313815f116ea99ae1864,78,381.574500,43.490418,204.5735,452.63,1
21,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,8.549667,stock,10.0,short_time_horizon,3d8664d2b058d4796589a4dbedcc1a48,42,10.971667,0.688024,10.0300,12.13,1
32,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,17.973450,stock,100.0,short_time_horizon,209cbf0d52efd985e9a15aad05ee1e3a,141,215.919787,121.385896,10.1000,698.64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5562,med_risk_tolerance,limited_investment_exp,not_important_liq_need,iOS,15.122067,stock,100.0,short_time_horizon,42d770c7fd58e1a3c93ff190cab9cbdc,59,1236.723051,901.003937,100.0000,3064.58,1
5566,high_risk_tolerance,good_investment_exp,very_important_liq_need,iOS,27.200983,stock,30.0,short_time_horizon,193b7e9382043cce5ae965268ec47879,170,146.787466,66.139954,46.8800,275.13,1
5571,med_risk_tolerance,extensive_investment_exp,very_important_liq_need,iOS,0.000000,stock,60.0,short_time_horizon,f415b4e8c761affcae0925dff482a12b,225,178.703912,157.664706,11.7400,496.33,1
5578,med_risk_tolerance,no_investment_exp,not_important_liq_need,iOS,13.606267,stock,10.0,short_time_horizon,abc394015ddbf4a8c641f65fb401ba0d,7,10.228786,0.108537,10.0100,10.31,1
